#  Organización de datos en lotes de entrenamiento

El siguiente paso es construir los lotes de entrenamiento de manera efectiva. Esto implica definir un método que garantice que el modelo reciba los datos de entrenamiento formateados durante el proceso de ajuste.

![Texto alternativo](./imgs/7.5.png)

En la sección 06,   los  lotes  de  entrenamiento  fueron  creados  automáticamente  por  la  clase  DataLoader  de  PyTorch ,  que  emplea  una  función  de  intercalación  predeterminada  para  combinar  listas  de  muestras  en  lotes.  Esta  función  se  encarga  de  tomar  una  lista  de muestras  de  datos  individuales  y  combinarlas  en  un  único  lote  que  el  modelo  puede  procesar  eficientemente  durante  el  entrenamiento.

Sin  embargo,  el  proceso  de  procesamiento  por  lotes  para  el  ajuste  fino  de  instrucciones  en  esta sección es  un  poco  más  complejo  y  requiere  la  creación  de  una  función  de intercalación  personalizada  que  posteriormente se conectará  al  DataLoader.

En  esta  sección,  se abordará  el  proceso  de  procesamiento  por  lotes  en  varios  pasos,  incluida  la  codificación de  la  función  de  intercalación  personalizada.

![Texto alternativo](./imgs/7.6.png)

Primero se implementan los pasos 2.1 y 2.2, se codifica una clase InstructionDataset que aplica format_input y pre_tokeniza todas las entradas en el conjunto de datos.

![Texto alternativo](./imgs/7.7.png)


In [ ]:
def format_input(entry):
    instruction_text = (
        f"Below is an instruction that describes a task. "
        f"Write a response that appropriately completes the request."
        f"\n\n### Instruction:\n{entry['instruction']}"
    )
    input_text = f"\n\n### Input:\n{entry['input']}" if entry["input"] else ""
    return instruction_text + input_text

In [2]:
import torch
from torch.utils.data import Dataset

class InstructionDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.encoded_text = []

        for entry in data:                                       
            instruction_plus_input = format_input(entry)
            response_text = f"\n\n### Response:\n{entry['output']}"
            full_text = instruction_plus_input + response_text
            self.encoded_texts.append(
                tokenizer.encode(full_text)
            )

    def __getitem__(self, index):
        return self.encoded_text[index]
    
    def __len__(self):
        return len(self.data)

Similar al enfoque de la sección 06, se busca acelerar el entrenamiento recopilando múltiples ejemplos de entrenamiento en un lote, lo que requiere rellenar todas las entradas con una logitud similar.  En  lugar  de  añadir  los  tokens  <|endoftext|>  a  las  entradas  de  texto,  se puede añadir  su  ID  de  token  directamente  a  las  entradas  pretokenizadas.  

En  la sección 06, se igualaba  la  longitud  de  todos  los  ejemplos  de  un  conjunto  de  datos.  Pasando  al  paso  2.3  de  la  figura, se adopta  un  enfoque  más  sofisticado:  desarrollar  una  función  de  intercalación  personalizada  que  se puede pasar  al  cargador  de  datos.  Esta  función  de  intercalación  personalizada  iguala  la  longitud  de  los  ejemplos  de  entrenamiento  de  cada  lote,  permitiendo  que  los  distintos  lotes  tengan  longitudes  diferentes.  Este  enfoque  minimiza  el  relleno  innecesario,  extendiendo  las  secuencias  solo  para  que  coincidan  con  la  más  larga  de  cada  lote,  no  con  todo  el  conjunto  de  datos.

![Texto alternativo](./imgs/7.8.png)

In [ ]:
def custom_collate_draft_1(batch, pad_token_id=50256, device='cpu'):
    #Calculamos las longitudes originales de cada secuencia
    batchs_lengths = [len(item) for item in batch]
    #Determinamos la longitud máxima del batch (+1 para añadir el token final)
    batch_max_length = max(batchs_lengths) + 1
    inputs_lst = []

    for item in batch:
        item += [pad_token_id]
        padded = item + [pad_token_id] * (batch_max_length - len(item))
        #Creamos el tensor de entrada quitando el último token ([:-1])
        #Esto se hace para alinear inputs y labels en el entrenamiento autoregresivo
        #El modelo ve "inputs" y debe predecir el siguiente token (label)
        #Por eso inputs = secuencia sin el último token
        #y labels  = secuencia sin el primero
        input = torch.tensor(padded[:-1])
        inputs_lst.append(input)
    
    inputs_tensor = torch.stack(inputs_lst).to(device)
    return inputs_tensor

inputs_1 = [0, 1, 2, 3, 4]
inputs_2 = [5, 6]
inputs_3 = [7, 8, 9]
batch = (
    inputs_1,
    inputs_2,
    inputs_3
)

print(custom_collate_draft_1(batch))

6
tensor([[    0,     1,     2,     3,     4],
        [    5,     6, 50256, 50256, 50256],
        [    7,     8,     9, 50256, 50256]])


Como se puede  ver  en  base  al  resultado  anterior,  todas  las  entradas  se  han  rellenado  hasta  la  longitud  dela  lista  de  entrada  más  larga,  inputs_1,  que  contiene  5  ID  de  token. Sin  embargo,  como  se vio en las secciones 05  y  06,  también  se necesitan  crear  lotes con  los  ID  de  token  de  destino,  correspondientes  al  lote  de  ID  de  entrada.  Estos  ID  de  destino, son  cruciales  porque  representan  lo  que quiere  que  el  modelo  genera y  lo  que  se necesita  durante  el  entrenamiento  para  calcular  la  pérdida  para  las  actualizaciones  de  peso.

![Texto alternativo](./imgs/7.9.png)


Similar  al  proceso  descrito  en la sección 05  para  el  preentrenamiento  de  un  LLM,  los  ID  de  los  tokens  de  destino  coinciden  con  los  de  entrada,  pero  se  desplazan  una  posición  a  la  derecha.  Esta  configuración,  como  se  muestra  en  la  figura,  permite  al  LLM  aprender  a  predecir  el  siguiente  token  en  una  secuencia.

![Texto alternativo](./imgs/7.10.png)

La  siguiente  función  de  intercalación  actualizada  genera  los  ID  de  token  de  destino,  a  partir  de  los  ID  de  token  de  entrada:

In [ ]:
def custom_collate_draft_2(batch,pad_token_id=50256,device="cpu"):
    batchs_lengths = [len(item) for item in batch]
    batch_max_length = max(batchs_lengths) + 1
    print(batch_max_length)
    inputs_lst, target_lst = [], []

    for item in batch:
        item += [pad_token_id]
        padded = item + [pad_token_id] * (batch_max_length - len(item))
        input = torch.tensor(padded[:-1]) #tensor de entradas
        target = torch.tensor(padded[1:]) #tensor de salidas
        inputs_lst.append(input)
        target_lst.append(target)
    
    inputs_tensor = torch.stack(inputs_lst).to(device)
    target_tensor = torch.stack(target_lst).to(device)
    return inputs_tensor, target_tensor

inputs_1 = [0, 1, 2, 3, 4]
inputs_2 = [5, 6]
inputs_3 = [7, 8, 9]
batch = (
    inputs_1,
    inputs_2,
    inputs_3
)
inputs, targets = custom_collate_draft_2(batch)
print(inputs)
print(targets)

6
tensor([[    0,     1,     2,     3,     4],
        [    5,     6, 50256, 50256, 50256],
        [    7,     8,     9, 50256, 50256]])
tensor([[    1,     2,     3,     4, 50256],
        [    6, 50256, 50256, 50256, 50256],
        [    8,     9, 50256, 50256, 50256]])
